In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv('final.csv', dtype=str)

In [4]:
data = df

In [5]:
df_nona = data.dropna(axis=1, how='all')

In [6]:
drop_columns = ['ActivityDepthHeightMeasure/MeasureValue', 
                'ActivityDepthHeightMeasure/MeasureUnitCode', 
                'ActivityDepthAltitudeReferencePointText', 
                'ActivityTopDepthHeightMeasure/MeasureValue', 
                'ActivityTopDepthHeightMeasure/MeasureUnitCode', 
                'ActivityBottomDepthHeightMeasure/MeasureValue',
                'ActivityBottomDepthHeightMeasure/MeasureUnitCode',
                'ActivityLocation/HorizontalCollectionMethodName',
                'ActivityLocation/HorizontalCoordinateReferenceSystemDatumName',
                'PassCount',
                'SampleCollectionMethod/MethodIdentifier',
                'SampleCollectionMethod/MethodIdentifierContext',
                'SampleCollectionMethod/MethodName',
                'SampleCollectionMethod/MethodQualifierTypeName',
                'SampleCollectionMethod/MethodDescriptionText',
                'SampleCollectionEquipmentName',
                'SampleCollectionMethod/SampleCollectionEquipmentCommentText',
                'SamplePreparationMethod/MethodIdentifier',
                'SamplePreparationMethod/MethodIdentifierContext',
                'SamplePreparationMethod/MethodName',
                'SampleContainerTypeName',
                'SampleContainerColorName',
                'ThermalPreservativeUsedName',
                'SampleTransportStorageDescription',
                'ActivityMetricUrl',
                'ProviderName',
                'ActivityMetricType/MetricTypeIdentifier',
                'ActivityMetricType/MetricTypeIdentifierContext',
                'MetricTypeCitation/ResourceTitleName',
                'MetricTypeCitation/ResourceCreatorName',
                'MetricTypeCitation/ResourceSubjectText',
                'MetricTypeCitation/ResourcePublisherName',
                'MetricTypeCitation/ResourceDate',
                'MetricTypeCitation/ResourceIdentifier',
                'MetricTypeCitation/FormulaDescriptionText',
                'AnalysisStartTime/Time',
                'AnalysisStartTime/TimeZoneCode',
                'ElectronicAddress',
                'Telephonic',
                'OrganizationAddress/AddressTypeName_1',
                'OrganizationAddress/AddressText_1',
                'OrganizationAddress/SupplementalAddressText_1',
                'OrganizationAddress/LocalityName_1',
                'OrganizationAddress/StateCode_1',
                'OrganizationAddress/PostalCode_1',
                'OrganizationAddress/CountryCode_1',
                'OrganizationAddress/CountyCode_1',
                'OrganizationAddress/AddressTypeName_2',
                'OrganizationAddress/AddressText_2',
                'OrganizationAddress/AddressText_2',
                'OrganizationAddress/LocalityName_2',
                'OrganizationAddress/StateCode_2',
                'OrganizationAddress/PostalCode_2',
                'OrganizationAddress/CountryCode_2',
                'OrganizationAddress/CountyCode_2',
                'OrganizationAddress/AddressTypeName_3',
                'OrganizationAddress/AddressText_3',
                'OrganizationAddress/SupplementalAddressText_3',
                'OrganizationAddress/LocalityName_3',
                'OrganizationAddress/StateCode_3',
                'OrganizationAddress/PostalCode_3',
                'OrganizationAddress/CountryCode_3',
                'OrganizationAddress/CountyCode_3',
                'SamplingDesignTypeCode'
               ]
df_columnclean = df_nona.drop(columns=drop_columns)
df_naclean = df_columnclean.dropna(subset=['ResultMeasureValue', 'CharacteristicName', 'ActivityStartDate'])

In [7]:
df_ts = df_naclean[['OrganizationIdentifier',
                    'ActivityStartDate', 
                    'CharacteristicName', 
                    'ResultSampleFractionText', 
                    'ResultMeasureValue', 
                    'ResultMeasure/MeasureUnitCode', 
                    'StatisticalBaseCode', 
                    'ResultValueTypeName', 
                    'ResultStatusIdentifier',
                    'MonitoringLocationIdentifier', 
                    'MonitoringLocationName', 
                    'ActivityLocation/LongitudeMeasure', 
                    'LongitudeMeasure', 
                    'ActivityLocation/LatitudeMeasure', 
                    'LatitudeMeasure']]

In [8]:
df_ts['ActivityStartDate'] = pd.to_datetime(df_ts['ActivityStartDate'])
timefiltered_df = df_ts[df_ts['ActivityStartDate'].dt.year >= 2010]
filtered_df = timefiltered_df[~timefiltered_df['OrganizationIdentifier'].isin(['NALMS', 'CHATTAHOOCHEERIVERKEEPER'])]
filtered_df = filtered_df.dropna(subset=['MonitoringLocationIdentifier'])
USGS_df = filtered_df[filtered_df['MonitoringLocationIdentifier'].str.startswith('USGS-')]
USGS_df = USGS_df[USGS_df['ResultStatusIdentifier'] == 'Accepted']
real_USGS_df = USGS_df[USGS_df['ResultValueTypeName'] == 'Actual']
real_USGS_df = real_USGS_df.drop(columns=['MonitoringLocationName', 'ActivityLocation/LongitudeMeasure', 'ActivityLocation/LatitudeMeasure'])

C:\Users\Fred\AppData\Local\Temp\ipykernel_2324\1672277112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ts['ActivityStartDate'] = pd.to_datetime(df_ts['ActivityStartDate'])


In [9]:
for index, row in real_USGS_df.iterrows():
    site_number = row['MonitoringLocationIdentifier']
    if len(site_number) == 20:
        latitude = float(site_number[5:7] + '.' + site_number[7:11])
        longitude = -float(site_number[12:14] + '.' + site_number[14:18])
        real_USGS_df.at[index, 'LatitudeMeasure'] = latitude
        real_USGS_df.at[index, 'LongitudeMeasure'] = longitude

In [10]:
numbers = set()
for line in real_USGS_df['MonitoringLocationIdentifier'].unique():
    number = re.findall(r'\d+', line)[0]
    numbers.add(number)
numbers = {num for num in numbers if len(num) < 14}

In [11]:
site_dict = {}
for site_number in numbers:
    url = f"https://waterdata.usgs.gov/nwis/inventory/?site_no={site_number}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        dd_tags = soup.find_all('dd')
        for dd in dd_tags:
            if "Latitude" in dd.text and "Longitude" in dd.text:
                split_text = dd.text.split()
                latitude = re.sub(r'\D', '', split_text[1])
                latitude = latitude[:2] + '.' + latitude[2:]
                longitude = re.sub(r'\D', '', split_text[3])
                longitude = longitude[:2] + '.' + longitude[2:]
                key = 'USGS-' + site_number
                site_dict[key] = {'Latitude': latitude, 'Longitude': longitude}
                break
    else:
        print(f"Failed to fetch data for site number: {site_number}")

In [12]:
for index, row in real_USGS_df.iterrows():
    site_number = row['MonitoringLocationIdentifier']
    if site_number in site_dict:
        real_USGS_df.at[index, 'LongitudeMeasure'] = -float(site_dict[site_number]['Longitude'])
        real_USGS_df.at[index, 'LatitudeMeasure'] = float(site_dict[site_number]['Latitude'])

In [13]:
filtered_df['LatitudeMeasure'] = pd.to_numeric(filtered_df['LatitudeMeasure'], errors='coerce')
filtered_df['LongitudeMeasure'] = pd.to_numeric(filtered_df['LongitudeMeasure'], errors='coerce')
filtered_df = filtered_df[(filtered_df['LatitudeMeasure'] < 33.87)]

In [14]:
real_USGS_df = real_USGS_df.drop(columns=['ResultValueTypeName', 'ResultStatusIdentifier', 'OrganizationIdentifier'])

In [15]:
real_USGS_df.to_csv('real_USGS_df.csv', index=False)

In [16]:
real_USGS_df

,ActivityStartDate,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,StatisticalBaseCode,MonitoringLocationIdentifier,LongitudeMeasure,LatitudeMeasure
2691,2017-11-16,Stream flow,NaN,805,ft3/s,Mean,USGS-02336000,-84.2716,33.5133
2692,2017-11-16,Stream flow,NaN,23,m3/sec,Mean,USGS-02336000,-84.2716,33.5133
2693,2017-11-16,Escherichia coli,NaN,40,MPN/100 ml,NaN,USGS-02336000,-84.2716,33.5133
2694,2017-11-16,Total Coliform,Total,2000,MPN/100 ml,NaN,USGS-02336000,-84.2716,33.5133
2723,2014-02-27,Escherichia coli,NaN,12,MPN/100 ml,NaN,USGS-02336000,-84.2716,33.5133
...,...,...,...,...,...,...,...,...,...
426495,2012-09-10,"Stream flow, instantaneous",NaN,0.00,m3/sec,NaN,USGS-340388084232801,-84.2328,34.0388
426496,2012-09-10,"Temperature, water",NaN,21.6,deg C,NaN,USGS-340388084232801,-84.2328,34.0388
426497,2012-09-10,"Stream flow, instantaneous",NaN,0.06,ft3/s,NaN,USGS-340388084232801,-84.2328,34.0388
426498,2012-09-10,Specific conductance,Total,39,uS/cm @25C,NaN,USGS-340388084232801,-84.2328,34.0388


In [17]:
grouped = real_USGS_df.groupby('MonitoringLocationIdentifier')

for name, group_df in grouped:
    group_df.to_csv(f'{name}.csv', index=False)